In [1]:
# enabling gmaps to run in jupyter notebook
!jupyter nbextension enable --py gmaps 
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
#Store Part I results into DataFrame
#Load the csv exported in Part I to a DataFrame
cities_data = pd.read_csv('Cities.csv')
cities_data.head()

,Unnamed: 0,city,latitude,longitude,temperature,windspeed,country,date,clouds,humidity
0,0,rikitea,-23.12,-134.97,73.33,13.44,PF,1603194851,13,78
1,1,zhezkazgan,47.80,67.71,59.02,6.98,KZ,1603194851,0,24
2,2,yellowknife,62.46,-114.35,25.00,5.01,CA,1603194851,83,79
3,3,huangchuan,32.13,115.04,61.03,3.00,CN,1603194851,100,70
4,4,new norfolk,-42.78,147.06,54.00,2.57,AU,1603194852,89,67


In [3]:
#Humidity Heatmap

#Configure gmaps.
#Use the Lat and Longitude as locations and Humidity as the weight.
#Add Heatmap layer to map.
#pull out lat and long
#will show where humdity is highest 
fig = gmaps.figure()
heat_map = gmaps.heatmap_layer(cities_data[['latitude','longitude']], weights = cities_data['humidity'])
fig.add_layer(heat_map)
fig


Figure(layout=FigureLayout(height='420px'))

In [4]:
# find city with my ideal weather between 70 and 80" F and 0 clouds
ideal_weather= cities_data.loc[(cities_data['temperature']>70)&(cities_data['temperature']<80)&(cities_data['windspeed']<10)&(cities_data['clouds']==0)].dropna()
ideal_weather

,Unnamed: 0,city,latitude,longitude,temperature,windspeed,country,date,clouds,humidity
18,18,celestun,20.87,-90.40,79.00,9.28,MX,1603194854,0,83
132,132,akdepe,42.06,59.38,78.80,6.93,TM,1603194875,0,12
183,183,cabo san lucas,22.89,-109.91,75.00,1.01,MX,1603194884,0,86
320,320,neyshabur,36.21,58.80,73.87,4.03,IR,1603194915,0,13
378,378,murgab,37.50,61.97,78.80,9.17,TM,1603194925,0,14
415,415,tamiahua,21.27,-97.45,74.14,2.19,MX,1603194933,0,87
425,425,komsomolskiy,40.43,71.72,73.40,6.93,UZ,1603194935,0,25
504,504,wadi maliz,36.47,8.55,71.60,3.36,TN,1603194950,0,40
506,506,tatawin,32.93,10.45,75.20,4.70,TN,1603194951,0,33


In [5]:
# locating hotels 5000 meters within my coordinates above (i.e. ideal weather)
# set parameters
hotel_df = ideal_weather[['city','country','latitude','longitude']].copy()
#creating a 5th column called hotel name to list hotel names
hotel_df['hotel name'] = ''
params= {
    'radius': 5000,
    #gmaps uses the word 'lodging' to find hotels
    'types': 'lodging',
    'key': g_key
}
# using the above find the nearest hotel
for index, city in hotel_df.iterrows():
    params['location']= f"{city['latitude']},{city['longitude']}"
    url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    #we will call this url and these are the parameters we shall get
    address=requests.get(url,params=params).json()
    try: 
        #we got the address. the .loc os putting in the hotel name. the address has results, array and name key
        hotel_df.loc[index,'hotel name']=address['results'][0]['name']
    except: 
        print(city['city']+' not found')
hotel_df



murgab not found


,city,country,latitude,longitude,hotel name
18,celestun,MX,20.87,-90.40,Posada Lilia
132,akdepe,TM,42.06,59.38,Мой дом
183,cabo san lucas,MX,22.89,-109.91,Hotel Tesoro Los Cabos
320,neyshabur,IR,36.21,58.80,Amiran Hotel
378,murgab,TM,37.50,61.97,
415,tamiahua,MX,21.27,-97.45,Mission Of The Sun Hotel
425,komsomolskiy,UZ,40.43,71.72,Ikathouse
504,wadi maliz,TN,36.47,8.55,HOPITAL
506,tatawin,TN,32.93,10.45,Sangho Privilege Tataouine


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [7]:
# create a marker map at each of the hotel locations
mark_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(mark_layer)
fig

Figure(layout=FigureLayout(height='420px'))